<a href="https://colab.research.google.com/github/Sai0472003/PatentLitigation_Impact/blob/main/Causal_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Loading and Cleaning

In [194]:
#loading pre-cleaned csv files
clean_plaintiff_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_RnD.csv')
clean_defendent_RnD = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_RnD.csv')

clean_plaintiff_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_plaintiff_npm.csv')
clean_defendent_npm = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/clean_defendent_npm.csv')

pd.set_option('display.max_columns', None)

In [195]:
#adding dummy variable for Defendant to all dfs
def_npm = clean_defendent_npm.copy()
def_npm['Defendant'] = 1
plt_npm = clean_plaintiff_npm.copy()
plt_npm['Defendant'] = 0

def_rnd = clean_defendent_RnD.copy()
def_rnd['Defendant'] = 1
plt_rnd = clean_plaintiff_RnD.copy()
plt_rnd['Defendant'] = 0

In [196]:
#concat merging to create two complete dfs for plaintiffs and defendants for npm/rnd
complete_npm = pd.concat([plt_npm, def_npm], axis=0)
complete_rnd = pd.concat([plt_rnd, def_rnd], axis=0)

In [197]:
#cleaning up/ensuring clean and accurate merge for npm

#filtering out some rows that don't benefit analysis
complete_npm = complete_npm[complete_npm['NOS'] == 830]
complete_npm = complete_npm[complete_npm['disp_transferred'] == 0]

#dropping useless columns
complete_npm = complete_npm.drop(columns=['PLT', 'case_row_id', 'gics_sector_55',
                                          'gics_sector_60', 'CLASSACT', 'DEF',
                                          'NOS', 'disp_transferred'])

complete_npm

,case_number,date_filed,date_closed,filing_quarter_npm,closing_quarter_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,delta_npm,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant
0,4:96-cv-02682,1996-09-03,1999-02-23,0.062715,0.077481,0.063360,0.062349,0.060794,0.067324,0.069235,0.073743,SC,0.014767,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,5:97-cv-03023,1997-11-17,2002-05-06,-0.062471,0.589367,-0.074369,-0.080820,-0.081557,0.584728,0.753730,0.765068,VA,0.651838,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2:95-cv-02147,1995-07-21,1995-10-04,0.067400,0.068283,0.066354,0.065138,0.063037,0.068659,0.068978,0.069198,AR,0.000883,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,5:12-cv-05142,2012-07-10,2012-12-03,0.052579,0.059432,0.050871,0.051547,0.067735,0.068362,0.068581,0.071595,MN,0.006853,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0
4,2:96-cv-00765,1996-03-28,1996-08-22,-0.000463,0.066122,0.021794,0.045216,0.042667,-0.229868,-0.267930,-0.289274,AZ,0.066585,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2821,3:99-cv-00802,2002-05-17,2002-07-24,-0.032404,-0.039621,-0.029730,-0.020944,-0.014458,-0.003570,-0.012436,-0.032884,FL,-0.007217,68,1,0,0,0,0,7,58,3,3,55025,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2822,3:15-cv-00621,2015-09-25,2022-05-11,0.228458,0.257090,0.226164,0.225335,0.222533,0.253096,0.245579,0.244929,CA,0.028632,2420,0,0,0,0,1,7,58,3,3,55025,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,1
2823,3:11-cv-00709,2011-10-17,2012-03-22,-0.035093,-0.009664,-0.034976,-0.038511,-0.041046,0.007859,0.041828,0.074071,MA,0.025429,157,0,0,0,0,1,7,58,3,3,88888,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0,1
2824,3:97-cv-00860,1997-12-18,1998-03-12,0.078352,0.079669,0.077721,0.083433,0.093009,0.083175,0.083890,0.085331,UT,0.001316,84,0,0,0,1,0,7,58,3,3,55025,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [198]:
#cleaning up/ensuring clean and accurate merge for rnd

complete_rnd = complete_rnd[complete_rnd['disp_transferred'] == 0]

#dropping useless columns
complete_rnd = complete_rnd.drop(columns=['PLT', 'case_row_id', 'gics_sector_55',
                                          'gics_sector_60', 'CLASSACT', 'DEF',
                                          'NOS', 'disp_transferred'])

complete_rnd

,case_number,date_filed,date_closed,filing_quarter_rnd,closing_quarter_rnd,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,delta_rnd_sales,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant
0,4:96-cv-02682,1996-09-03,1999-02-23,0.004627,0.005521,0.004655,0.004652,0.004693,0.005580,0.005550,0.004712,SC,0.000894,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,5:97-cv-03023,1997-11-17,2002-05-06,0.087230,0.101858,0.090111,0.092890,0.095311,0.103234,0.101100,0.100825,VA,0.014628,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2:95-cv-02147,1995-07-21,1995-10-04,0.031698,0.036356,0.032723,0.034003,0.035394,0.035828,0.035236,0.034874,AR,0.004657,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,5:12-cv-05142,2012-07-10,2012-12-03,0.008999,0.015458,0.010234,0.011822,0.014060,0.015378,0.014903,0.014365,MN,0.006459,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0
4,2:96-cv-00765,1996-03-28,1996-08-22,0.284530,0.126727,0.279593,0.270024,0.273538,0.438345,0.454380,0.477555,AZ,-0.157803,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,3:99-cv-00802,2002-05-17,2002-07-24,0.023790,0.024978,0.022394,0.020589,0.023064,0.017403,0.016724,0.016442,FL,0.001188,68,1,0,0,0,0,7,58,3,3,55025,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2831,3:15-cv-00621,2015-09-25,2022-05-11,0.034516,0.065185,0.033579,0.033474,0.033063,0.066571,0.071358,0.074589,CA,0.030669,2420,0,0,0,0,1,7,58,3,3,55025,0,0,0,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,1
2832,3:11-cv-00709,2011-10-17,2012-03-22,0.174217,0.170799,0.174914,0.174777,0.174412,0.163650,0.158431,0.155543,MA,-0.003418,157,0,0,0,0,1,7,58,3,3,88888,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0,1
2833,3:97-cv-00860,1997-12-18,1998-03-12,0.071796,0.070088,0.072638,0.071312,0.069895,0.069579,0.069070,0.069014,UT,-0.001708,84,0,0,0,1,0,7,58,3,3,55025,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1


In [199]:
#identifying unique case numbers across both dataframes?
known_cases = list(set(complete_rnd['case_number'].unique()).intersection(set(complete_npm['case_number'].unique())))

In [200]:
len(known_cases)

3663

In [201]:
#loading up the attorneys csv and filtering for known cases in the dataset
attorneys = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/attorneys.csv')

#filtering out unecessary values
attorneys = attorneys[attorneys['case_number'].isin(known_cases)]
attorneys = attorneys[attorneys['party_type'].isin(['Defendant', 'Plaintiff'])]

#only keeping lead attorneys for ease
attorneys = attorneys[attorneys['position'].isin(['LEAD ATTORNEY; ATTORNEY TO BE NOTICED',
                                                  'LEAD ATTORNEY; ATTORNEY TO BE NOTICED; PRO HAC VICE',
                                                  'PRO HAC VICE; LEAD ATTORNEY; ATTORNEY TO BE NOTICED',
                                                  'LEAD ATTORNEY; PRO HAC VICE; ATTORNEY TO BE NOTICED',
                                                  'LEAD ATTORNEY; ATTORNEY TO BE NOTICED; Bar Status: Admitted/In Good Standing',
                                                  'LEAD ATTORNEY'])]

#creating a case number index for ease
attorneys['case_no'] = attorneys['case_number']
attorneys = attorneys.set_index(['case_no']).sort_index()

#creating dummy for
attorneys['Defendant'] = 0
attorneys['Defendant'][attorneys['party_type'] == 'Defendant'] = 1

#dropping irrelavent columns
attorneys = attorneys.drop(columns=['district_id', 'case_number_raw', 'case_row_id',
                                    'party_row_count', 'party_type',
                                    'attorney_contactinfo', 'attorney_row_count'])

attorneys = attorneys.drop_duplicates()
attorneys

,case_number,attorney_name,position,Defendant
case_no,,,,
0:11-cv-00569,0:11-cv-00569,Rachel K. Zimmerman,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,1
0:11-cv-00569,0:11-cv-00569,Heather J Kliebenstein,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,1
0:11-cv-00569,0:11-cv-00569,Allen W Hinderaker,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,1
0:11-cv-00569,0:11-cv-00569,Paige S Stradley,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,1
0:11-cv-00569,0:11-cv-00569,Tong Wu,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,1
...,...,...,...,...
9:17-cv-80886,9:17-cv-80886,Ernesto Miguel Rubi,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,0
9:17-cv-80886,9:17-cv-80886,Amy Michelle Bowers,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,0
9:17-cv-80886,9:17-cv-80886,John C. Carey,LEAD ATTORNEY; ATTORNEY TO BE NOTICED,0


In [202]:
# Group by case_number and filter groups that have both a defendant and non-defendant
filtered = attorneys.groupby('case_number').filter(
    lambda g: (g['Defendant'] == 1).any() and (g['Defendant'] == 0).any()
)

# For each case_number, pick one row with Defendant==1 and one with Defendant==0
def get_pair(group):
    return pd.concat([
        group[group['Defendant'] == 1].head(1),
        group[group['Defendant'] == 0].head(1)
    ])

clean_attorneys = filtered.groupby('case_number').apply(get_pair).reset_index(drop=True)

In [203]:
clean_attorneys['attorney_name'].value_counts().head(25).sum()

np.int64(600)

In [204]:
complete_npm = complete_npm.merge(clean_attorneys[['case_number', 'Defendant', 'attorney_name']], on=['case_number', 'Defendant'], how='left')
complete_rnd = complete_rnd.merge(clean_attorneys[['case_number', 'Defendant', 'attorney_name']], on=['case_number', 'Defendant'], how='left')

complete_npm.head()

,case_number,date_filed,date_closed,filing_quarter_npm,closing_quarter_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,delta_npm,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant,attorney_name
0,4:96-cv-02682,1996-09-03,1999-02-23,0.062715,0.077481,0.063360,0.062349,0.060794,0.067324,0.069235,0.073743,SC,0.014767,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,"Harold Simmons Tate, Jr"
1,5:97-cv-03023,1997-11-17,2002-05-06,-0.062471,0.589367,-0.074369,-0.080820,-0.081557,0.584728,0.753730,0.765068,VA,0.651838,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2,2:95-cv-02147,1995-07-21,1995-10-04,0.067400,0.068283,0.066354,0.065138,0.063037,0.068659,0.068978,0.069198,AR,0.000883,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
3,5:12-cv-05142,2012-07-10,2012-12-03,0.052579,0.059432,0.050871,0.051547,0.067735,0.068362,0.068581,0.071595,MN,0.006853,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0,Kimberly N. Van Voorhis
4,2:96-cv-00765,1996-03-28,1996-08-22,-0.000463,0.066122,0.021794,0.045216,0.042667,-0.229868,-0.267930,-0.289274,AZ,0.066585,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN


In [205]:
complete_rnd.head()

,case_number,date_filed,date_closed,filing_quarter_rnd,closing_quarter_rnd,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,delta_rnd_sales,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant,attorney_name
0,4:96-cv-02682,1996-09-03,1999-02-23,0.004627,0.005521,0.004655,0.004652,0.004693,0.005580,0.005550,0.004712,SC,0.000894,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,"Harold Simmons Tate, Jr"
1,5:97-cv-03023,1997-11-17,2002-05-06,0.087230,0.101858,0.090111,0.092890,0.095311,0.103234,0.101100,0.100825,VA,0.014628,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2,2:95-cv-02147,1995-07-21,1995-10-04,0.031698,0.036356,0.032723,0.034003,0.035394,0.035828,0.035236,0.034874,AR,0.004657,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
3,5:12-cv-05142,2012-07-10,2012-12-03,0.008999,0.015458,0.010234,0.011822,0.014060,0.015378,0.014903,0.014365,MN,0.006459,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0,Kimberly N. Van Voorhis
4,2:96-cv-00765,1996-03-28,1996-08-22,0.284530,0.126727,0.279593,0.270024,0.273538,0.438345,0.454380,0.477555,AZ,-0.157803,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN


In [206]:
BERT_Predictions = pd.read_csv('/content/drive/My Drive/ECO225_Final_Project/DATA/BERT_Predictions.csv')
BERT_Predictions = BERT_Predictions[['case_number', 'BERT_Logits']]

In [207]:
complete_npm = complete_npm.merge(BERT_Predictions, on=['case_number'], how='inner')
complete_rnd = complete_rnd.merge(BERT_Predictions, on=['case_number'], how='inner')

complete_npm.head()

,case_number,date_filed,date_closed,filing_quarter_npm,closing_quarter_npm,minus_one_npm,minus_two_npm,minus_three_npm,plus_one_npm,plus_two_npm,plus_three_npm,state_province,delta_npm,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant,attorney_name,BERT_Logits
0,4:96-cv-02682,1996-09-03,1999-02-23,0.062715,0.077481,0.063360,0.062349,0.060794,0.067324,0.069235,0.073743,SC,0.014767,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,"Harold Simmons Tate, Jr",1.236328
1,5:97-cv-03023,1997-11-17,2002-05-06,-0.062471,0.589367,-0.074369,-0.080820,-0.081557,0.584728,0.753730,0.765068,VA,0.651838,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.380127
2,2:95-cv-02147,1995-07-21,1995-10-04,0.067400,0.068283,0.066354,0.065138,0.063037,0.068659,0.068978,0.069198,AR,0.000883,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,1.236328
3,5:12-cv-05142,2012-07-10,2012-12-03,0.052579,0.059432,0.050871,0.051547,0.067735,0.068362,0.068581,0.071595,MN,0.006853,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0,Kimberly N. Van Voorhis,0.388184
4,2:96-cv-00765,1996-03-28,1996-08-22,-0.000463,0.066122,0.021794,0.045216,0.042667,-0.229868,-0.267930,-0.289274,AZ,0.066585,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1.236328


In [208]:
complete_rnd.head()

,case_number,date_filed,date_closed,filing_quarter_rnd,closing_quarter_rnd,minus_one_rnd,minus_two_rnd,minus_three_rnd,plus_one_rnd,plus_two_rnd,plus_three_rnd,state_province,delta_rnd_sales,case_length,materials,industrials,consumer_discretionary,info_tech,health_care,CIRCUIT,DISTRICT,OFFICE,JURIS,COUNTY,disp_before_issue_joined,disp_dismissed,disp_settled,disp_judgment,judgment_monetary_award,judgment_injunction,judgment_ordered_loss,judgment_for_plaintiff,judgment_for_defendent,judgment_for_both,jury_demand_plaintiff,jury_demand_defendent,jury_demand_both,infringement_35_271,Defendant,attorney_name,BERT_Logits
0,4:96-cv-02682,1996-09-03,1999-02-23,0.004627,0.005521,0.004655,0.004652,0.004693,0.005580,0.005550,0.004712,SC,0.000894,903,1,0,0,0,0,4,20,4,3,45031,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,"Harold Simmons Tate, Jr",1.236328
1,5:97-cv-03023,1997-11-17,2002-05-06,0.087230,0.101858,0.090111,0.092890,0.095311,0.103234,0.101100,0.100825,VA,0.014628,1631,0,0,0,0,1,11,26,5,3,1089,0,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,0.380127
2,2:95-cv-02147,1995-07-21,1995-10-04,0.031698,0.036356,0.032723,0.034003,0.035394,0.035828,0.035236,0.034874,AR,0.004657,75,0,1,0,0,0,8,61,2,3,5131,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,1.236328
3,5:12-cv-05142,2012-07-10,2012-12-03,0.008999,0.015458,0.010234,0.011822,0.014060,0.015378,0.014903,0.014365,MN,0.006459,146,1,0,0,0,0,8,61,5,3,5141,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1,0,Kimberly N. Van Voorhis,0.388184
4,2:96-cv-00765,1996-03-28,1996-08-22,0.284530,0.126727,0.279593,0.270024,0.273538,0.438345,0.454380,0.477555,AZ,-0.157803,147,0,1,0,0,0,9,70,2,3,4013,1,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,NaN,1.236328


# Statistical Analysis: Fitting Models

In [211]:
#simple one predictor ols for npm
import statsmodels.api as sm

X = complete_npm['BERT_Logits']
y = complete_npm['disp_dismissed']

ols = sm.OLS(y, X).fit()
print(ols.summary())

                                 OLS Regression Results                                
Dep. Variable:         disp_dismissed   R-squared (uncentered):                   0.680
Model:                            OLS   Adj. R-squared (uncentered):              0.679
Method:                 Least Squares   F-statistic:                              7782.
Date:                Tue, 15 Apr 2025   Prob (F-statistic):                        0.00
Time:                        19:44:45   Log-Likelihood:                         -2703.2
No. Observations:                3671   AIC:                                      5408.
Df Residuals:                    3670   BIC:                                      5415.
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------

In [ ]:
from statsmodels.api import OLS, add_constant


X = complete_npm['disp_dismissed']
Z = complete_npm['BERT_Logits']
y = complete_npm['disp_dismissed']

# First stage: regress D on Z + controls
first_stage = OLS(X ~ Z)).fit()
df['D_hat'] = first_stage.fittedvalues

# Second stage: regress Y on D_hat + controls
second_stage = OLS(df['Y'], add_constant(df[['D_hat'] + X_controls])).fit()
print(second_stage.summary())

In [214]:
from linearmodels.iv import IV2SLS
from statsmodels.api import add_constant

Y = complete_npm['delta_npm']
D = complete_npm['disp_dismissed']
Z = complete_npm['BERT_Logits']
# The following line is changed to select individual columns as Series instead of a DataFrame
W = complete_npm[['case_length', 'info_tech', 'health_care']]

# Creating the DataFrame using the individual Series
df = pd.DataFrame({'Y': Y, 'D': D, 'Z': Z, 'case_length':W['case_length'], 'info_tech': W['info_tech'], 'health_care':W['health_care']})
df['const'] = 1

# Adjusting the formula to include individual exogenous variables
iv_model = IV2SLS.from_formula(
    'Y ~ 1 + case_length + info_tech + health_care + [D ~ Z]', data=df
).fit()

print(iv_model.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                      Y   R-squared:                     -0.0254
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0265
No. Observations:                3671   F-statistic:                    5.1625
Date:                Tue, Apr 15 2025   P-value (F-stat)                0.2710
Time:                        19:47:22   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
Intercept      -1.6537     0.7527    -2.1970     0.0280     -3.1291     -0.1784
case_length     0.0002     0.0002     1.3948    

In [219]:
complete_rnd['date_closed'].describe()

,date_closed
count,3692
unique,2410
top,2017-11-02
freq,8
